In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Datanauts Load").getOrCreate()

df = spark.read.csv("/content/drive/MyDrive/Datanauts_dataset.csv",
                    header=True, inferSchema=True)

df.show(5)

+----------+-----------+-------+----------+-------------------+-------------+-------------+--------------------+--------------------+--------------------+-----------+----------+----------------+-----------+----------------------+------------+----------------------------+------------+---------+-------------+--------+----+
|Offense ID|Agency Name| County|Population|      Location Name|Incident Date|Incident Hour|        Offense Name|    Offense Category|         Weapon Name|Victim Type|Victim Sex|Victim Age Group|Victim Race|Victim Resident Status|Offender Sex|Victim-Offender Relationship|Offender Age|Victim Id|Offender Race|   State|Year|
+----------+-----------+-------+----------+-------------------+-------------+-------------+--------------------+--------------------+--------------------+-----------+----------+----------------+-----------+----------------------+------------+----------------------------+------------+---------+-------------+--------+----+
| 181021371|      ADAMS|  ADAMS

In [3]:
df.columns

['Offense ID',
 'Agency Name',
 'County',
 'Population',
 'Location Name',
 'Incident Date',
 'Incident Hour',
 'Offense Name',
 'Offense Category',
 'Weapon Name',
 'Victim Type',
 'Victim Sex',
 'Victim Age Group',
 'Victim Race',
 'Victim Resident Status',
 'Offender Sex',
 'Victim-Offender Relationship',
 'Offender Age',
 'Victim Id',
 'Offender Race',
 'State',
 'Year']

In [4]:
# Sanitize column names
def sanitize(name):
    return (
        name.strip()
            .replace(" ", "_")
            .replace("/", "_")
            .replace("(", "")
            .replace(")", "")
            .replace("-", "_")
            .replace(".", "")
    )

for col in df.columns:
    df = df.withColumnRenamed(col, sanitize(col))

print("Columns after sanitizing:", df.columns)

Columns after sanitizing: ['Offense_ID', 'Agency_Name', 'County', 'Population', 'Location_Name', 'Incident_Date', 'Incident_Hour', 'Offense_Name', 'Offense_Category', 'Weapon_Name', 'Victim_Type', 'Victim_Sex', 'Victim_Age_Group', 'Victim_Race', 'Victim_Resident_Status', 'Offender_Sex', 'Victim_Offender_Relationship', 'Offender_Age', 'Victim_Id', 'Offender_Race', 'State', 'Year']


In [5]:
# Offender Age Diversity (Entropy)

from pyspark.sql import Window
from pyspark.sql import functions as F

df = df.withColumn(
    "Offender_Age_Group",
    F.when(F.col("Offender_Age") < 18, "Under 18")
     .when((F.col("Offender_Age") >= 18) & (F.col("Offender_Age") <= 25), "18-25")
     .when((F.col("Offender_Age") > 25) & (F.col("Offender_Age") <= 40), "26-40")
     .when((F.col("Offender_Age") > 40) & (F.col("Offender_Age") <= 60), "41-60")
     .otherwise("60+")
)

age_counts = (
    df.groupBy("Offense_Category", "Offender_Age_Group")
      .agg(F.count("*").alias("count"))
)

w = Window.partitionBy("Offense_Category").orderBy(F.desc("count"))

top_age_group = (
    age_counts
    .withColumn("rank", F.row_number().over(w))
    .filter("rank = 1")
    .orderBy("Offense_Category")
)

top_age_group.show(100, truncate=False)



+----------------------------------------+------------------+-------+----+
|Offense_Category                        |Offender_Age_Group|count  |rank|
+----------------------------------------+------------------+-------+----+
|Arson                                   |26-40             |3836   |1   |
|Assault Offenses                        |26-40             |1389934|1   |
|Bribery                                 |26-40             |487    |1   |
|Burglary/Breaking & Entering            |Under 18          |106863 |1   |
|Counterfeiting/Forgery                  |Under 18          |13879  |1   |
|Destruction/Damage/Vandalism of Property|Under 18          |226168 |1   |
|Embezzlement                            |26-40             |2105   |1   |
|Extortion/Blackmail                     |Under 18          |6394   |1   |
|Fraud Offenses                          |Under 18          |100454 |1   |
|Homicide Offenses                       |26-40             |5362   |1   |
|Human Trafficking       

- **Adults (26–40)** commit most violent and serious crimes such as Assault, Homicide, Sex Offenses, Kidnapping, Trafficking, and Arson.
- **Juveniles (Under 18)** dominate property and opportunistic crimes, including Burglary, Larceny/Theft, Motor Vehicle Theft, Robbery, Vandalism, and Fraud.
- White-collar crimes like Bribery and Embezzlement are mostly committed by 26–40-year-olds, reflecting workplace and financial access.
- **Young adults (18–25)** lead mainly in Sex Offenses (Non-forcible) involving close-age relationships.


In [6]:
# Gender Breakdown of Crime Types in the Dataset

from pyspark.sql import functions as F

df_sex = df.filter(F.col("Offender_Sex").isin("M", "F"))

sex_counts = (
    df_sex.groupBy("Offense_Category", "Offender_Sex")
          .agg(F.count("*").alias("count"))
)

pivot_sex = (
    sex_counts.groupBy("Offense_Category")
              .pivot("Offender_Sex", ["M", "F"])
              .sum("count")
              .na.fill(0)
)

pivot_sex = (
    pivot_sex.withColumn("Total", F.col("M") + F.col("F"))
             .withColumn("Male_Percent", F.round(F.col("M") / F.col("Total") * 100, 2))
             .withColumn("Female_Percent", F.round(F.col("F") / F.col("Total") * 100, 2))
)

pivot_sex.orderBy(F.desc("Male_Percent")).show(50, truncate=False)

+----------------------------------------+-------+------+-------+------------+--------------+
|Offense_Category                        |M      |F     |Total  |Male_Percent|Female_Percent|
+----------------------------------------+-------+------+-------+------------+--------------+
|Sex Offenses                            |177866 |10015 |187881 |94.67       |5.33          |
|Sex Offenses, Non-forcible              |4855   |376   |5231   |92.81       |7.19          |
|Robbery                                 |182054 |23694 |205748 |88.48       |11.52         |
|Homicide Offenses                       |12808  |1667  |14475  |88.48       |11.52         |
|Kidnapping/Abduction                    |41865  |6950  |48815  |85.76       |14.24         |
|Human Trafficking                       |2537   |453   |2990   |84.85       |15.15         |
|Burglary/Breaking & Entering            |155512 |34579 |190091 |81.81       |18.19         |
|Motor Vehicle Theft                     |128163 |31818 |159

- Male victims dominate across all offense categories.
- Crimes with violence, coercion, or confrontation show the highest male victimization.
- White-collar and financial crimes have the most balanced gender distribution.
- No category has a female majority.

In [7]:
# Crime Distribution by Time of Day
from pyspark.sql import functions as F

df_time = df.withColumn(
    "Hour_Range",
    F.when(F.col("Incident_Hour").between(0, 5), "00-05 Night")
     .when(F.col("Incident_Hour").between(6, 11), "06-11 Morning")
     .when(F.col("Incident_Hour").between(12, 17), "12-17 Afternoon")
     .otherwise("18-23 Evening")
)

state_hour_counts = (
    df_time.groupBy("State", "Hour_Range")
           .agg(F.count("*").alias("incidents"))
           .orderBy("State", "Hour_Range")
)

state_totals = df.groupBy("State").agg(F.count("*").alias("total_incidents"))

state_hour_counts = (
    state_hour_counts.join(state_totals, "State")
                     .withColumn("percent", F.round(F.col("incidents") / F.col("total_incidents") * 100, 2))
                     .drop("total_incidents")
)

state_hour_counts.show(100, truncate=False)

+----------+---------------+---------+-------+
|State     |Hour_Range     |incidents|percent|
+----------+---------------+---------+-------+
|Colorado  |06-11 Morning  |137133   |19.38  |
|Colorado  |00-05 Night    |148727   |21.02  |
|Colorado  |12-17 Afternoon|213944   |30.24  |
|Colorado  |18-23 Evening  |207700   |29.36  |
|NY        |00-05 Night    |223299   |17.43  |
|NY        |06-11 Morning  |256712   |20.04  |
|NY        |12-17 Afternoon|411744   |32.14  |
|NY        |18-23 Evening  |389199   |30.38  |
|New Mexico|18-23 Evening  |177398   |29.57  |
|New Mexico|00-05 Night    |101265   |16.88  |
|New Mexico|06-11 Morning  |119513   |19.92  |
|New Mexico|12-17 Afternoon|201824   |33.64  |
|Texas     |18-23 Evening  |1055077  |31.16  |
|Texas     |12-17 Afternoon|1069573  |31.59  |
|Texas     |00-05 Night    |659003   |19.46  |
|Texas     |06-11 Morning  |602610   |17.8   |
|Washington|06-11 Morning  |180812   |20.82  |
|Washington|12-17 Afternoon|272090   |31.33  |
|Washington|1

- Across all states, crime is highest in the **afternoon (12–5 PM)** and **evening (6–11 PM)**.  
- The **night hours (12–5 AM)** consistently show the lowest crime activity.  
- Colorado and New Mexico follow a similar pattern, while Texas and NY have especially high afternoon/evening crime levels.  

In [8]:
# Crime Distribution Across Seasons
from pyspark.sql import functions as F

df = df.withColumn("Incident_Date", F.to_date("Incident_Date"))

df = df.withColumn("Month", F.month("Incident_Date"))

df = df.withColumn(
    "Season",
    F.when(F.col("Month").isin(12, 1, 2), "Winter")
     .when(F.col("Month").isin(3, 4, 5), "Spring")
     .when(F.col("Month").isin(6, 7, 8), "Summer")
     .when(F.col("Month").isin(9, 10, 11), "Fall")
     .otherwise("Unknown")
)

season_count = df.groupBy("Season").count().orderBy(F.desc("count"))

season_count.show()


+------+-------+
|Season|  count|
+------+-------+
|Summer|1811136|
|  Fall|1751241|
|Spring|1708840|
|Winter|1572086|
+------+-------+





*  Crime is most frequent during Summer and Spring, possibly due to more outdoor activity and longer daylight hours.
*   Winter shows the lowest criminal activity, likely due to colder weather and shorter days.


In [9]:
# How Crime Has Changed Over the Years
from pyspark.sql import functions as F
from pyspark.sql.window import Window

df = df.withColumn("Incident_Date", F.to_date("Incident_Date"))

df = df.withColumn("Year_parsed", F.year("Incident_Date"))

yearly_counts = (
    df.groupBy("Year_parsed")
      .agg(F.count("*").alias("incidents"))
      .orderBy("Year_parsed")
)

w = Window.orderBy("Year_parsed")

yearly_counts = yearly_counts.withColumn(
    "prev_year_incidents", F.lag("incidents").over(w)
).withColumn(
    "change", F.col("incidents") - F.col("prev_year_incidents")
).withColumn(
    "percent_change", F.round(F.col("change") / F.col("prev_year_incidents") * 100, 2)
)

yearly_counts.show()

+-----------+---------+-------------------+------+--------------+
|Year_parsed|incidents|prev_year_incidents|change|percent_change|
+-----------+---------+-------------------+------+--------------+
|       2020|   889891|               NULL|  NULL|          NULL|
|       2021|  1179178|             889891|289287|         32.51|
|       2022|  1273717|            1179178| 94539|          8.02|
|       2023|  1784392|            1273717|510675|         40.09|
|       2024|  1716125|            1784392|-68267|         -3.83|
+-----------+---------+-------------------+------+--------------+



- **Overall Trend:** Crime increased substantially from **2020 to 2023**.
- **Peak Growth Years:** **2021** and **2023** show the largest spikes, likely reflecting social or policy changes.
- **Stabilization:** **2024** shows almost no increase, suggesting crime rates may be plateauing.

In [10]:
# Top Offense Categories Showing the Largest Yearly Increases
from pyspark.sql import functions as F
from pyspark.sql.window import Window

offense_yearly = (
    df.groupBy("Year_parsed", "Offense_Category")
      .agg(F.count("*").alias("incidents"))
      .orderBy("Offense_Category", "Year_parsed")
)

w = Window.partitionBy("Offense_Category").orderBy("Year_parsed")
offense_yearly = offense_yearly.withColumn(
    "prev_year_incidents", F.lag("incidents").over(w)
)

offense_yearly = offense_yearly.withColumn(
    "change", F.col("incidents") - F.col("prev_year_incidents")
).withColumn(
    "percent_change", F.round(F.col("change") / F.col("prev_year_incidents") * 100, 2)
)

offense_summary = (
    offense_yearly.groupBy("Offense_Category")
                  .agg(F.sum("change").alias("total_change"))
                  .orderBy(F.desc("total_change"))
)
offense_summary.show(10, truncate=False)

+----------------------------------------+------------+
|Offense_Category                        |total_change|
+----------------------------------------+------------+
|Assault Offenses                        |408476      |
|Larceny/Theft Offenses                  |180133      |
|Destruction/Damage/Vandalism of Property|95706       |
|Motor Vehicle Theft                     |43836       |
|Burglary/Breaking & Entering            |28638       |
|Fraud Offenses                          |18664       |
|Sex Offenses                            |15411       |
|Robbery                                 |11683       |
|Stolen Property Offenses                |10443       |
|Counterfeiting/Forgery                  |4945        |
+----------------------------------------+------------+
only showing top 10 rows



- Violent crimes like Assault and Robbery show significant upward trends, highlighting areas for law enforcement focus.
- Property crimes are also steadily increasing, suggesting preventive measures in communities may be needed.
- Fraud and financial crimes are rising moderately, indicating a potential shift towards economic or online crime trends.

In [11]:

# Firearm Involvement Rate per Offense Category

firearm = df.filter(F.lower(F.col("Weapon_Name")).rlike("gun|firearm|rifle|shotgun|pistol"))

firearm_cat = firearm.groupBy("Offense_Category").count() \
                     .withColumnRenamed("count","firearm_incidents")

total_cat = df.groupBy("Offense_Category").count().withColumnRenamed("count", "total_incidents")

firearm_rate = firearm_cat.join(total_cat, "Offense_Category") \
                          .withColumn("firearm_rate",
                                      F.round(F.col("firearm_incidents") / F.col("total_incidents") * 100, 2)) \
                          .orderBy(F.desc("firearm_rate"))

firearm_rate.show(30, truncate=False)


+--------------------+-----------------+---------------+------------+
|Offense_Category    |firearm_incidents|total_incidents|firearm_rate|
+--------------------+-----------------+---------------+------------+
|Homicide Offenses   |11172            |16261          |68.7        |
|Robbery             |95977            |221743         |43.28       |
|Kidnapping/Abduction|4974             |49433          |10.06       |
|Assault Offenses    |269226           |3593276        |7.49        |
|Human Trafficking   |169              |3174           |5.32        |
|Extortion/Blackmail |226              |11387          |1.98        |
|Sex Offenses        |2074             |193210         |1.07        |
+--------------------+-----------------+---------------+------------+



- Firearms are most commonly associated with homicides and robberies, emphasizing their role in serious violent crimes.
- In most other offenses (assault, trafficking, sex offenses), firearms are less frequently involved, suggesting other methods or weapons are more common.


In [12]:
# States with highest weapon-related crimes

weapon_flag = df.filter(
    (F.col("Weapon_Name").isNotNull()) &
    (F.col("Weapon_Name") != "") &
    (F.lower(F.col("Weapon_Name")) != "unknown")
)

state_total = df.groupBy("State").agg(F.count("*").alias("total_incidents"))
state_weapon = weapon_flag.groupBy("State").agg(F.count("*").alias("weapon_incidents"))

state_weapon_counts = (
    state_total.join(state_weapon, "State", "left")
               .na.fill(0)
               .withColumn("weapon_percent",
                           F.round(F.col("weapon_incidents") / F.col("total_incidents") * 100, 2))
               .orderBy(F.desc("weapon_incidents"))
)
state_weapon_counts.show(50)


+----------+---------------+----------------+--------------+
|     State|total_incidents|weapon_incidents|weapon_percent|
+----------+---------------+----------------+--------------+
|     Texas|        3386263|         3357219|         99.14|
|        NY|        1280954|         1271750|         99.28|
|Washington|         868582|          862115|         99.26|
|  Colorado|         707504|          698695|         98.75|
|New Mexico|         600000|          592334|         98.72|
+----------+---------------+----------------+--------------+



- Almost all reported crimes in these states involve a weapon, highlighting the prevalence of weapon use in criminal activity.
- Texas, NY, and Washington not only have high total incidents but also very high weapon involvement, indicating areas where weapon control and crime prevention measures could be prioritized.

In [13]:
# What are the top 5 weapons used in each state?

from pyspark.sql import functions as F
from pyspark.sql.window import Window

weapon_counts = (
    df.groupBy("State", "Weapon_Name")
      .agg(F.count("*").alias("cnt"))
)

w = Window.partitionBy("State").orderBy(F.desc("cnt"))

top5_weapons = (
    weapon_counts
    .withColumn("rank", F.row_number().over(w))
    .filter("rank <= 3")
    .orderBy("State", "rank")
)

top5_weapons.show(200, truncate=False)


+----------+------------------------+-------+----+
|State     |Weapon_Name             |cnt    |rank|
+----------+------------------------+-------+----+
|Colorado  |Personal Weapons        |587862 |1   |
|Colorado  |Handgun                 |35335  |2   |
|Colorado  |Other                   |22367  |3   |
|NY        |Personal Weapons        |1160408|1   |
|NY        |Knife/Cutting Instrument|40740  |2   |
|NY        |Other                   |30174  |3   |
|New Mexico|Personal Weapons        |486987 |1   |
|New Mexico|Handgun                 |24900  |2   |
|New Mexico|Other                   |22332  |3   |
|Texas     |Personal Weapons        |2837527|1   |
|Texas     |Handgun                 |145044 |2   |
|Texas     |Other                   |120214 |3   |
|Washington|Personal Weapons        |774965 |1   |
|Washington|Other                   |25919  |2   |
|Washington|Handgun                 |18606  |3   |
+----------+------------------------+-------+----+



In [14]:
# Count incidents per location & category

from pyspark.sql import Window
import pyspark.sql.functions as F


loc_dist = df.groupBy("Offense_Category", "Location_Name").count()

total = df.groupBy("Offense_Category").count().withColumnRenamed("count","total")

loc_join = loc_dist.join(total, "Offense_Category") \
                   .withColumn("percent", F.round(F.col("count")/F.col("total") * 100, 2))

w = Window.partitionBy("Offense_Category").orderBy(F.desc("count"))
loc_rank = loc_join.withColumn("rank", F.row_number().over(w))

top_2_loc = loc_rank.filter(F.col("rank") <= 1).orderBy("Offense_Category", "rank")
top_2_loc.show(200, truncate=False)


+----------------------------------------+----------------------------------+-------+-------+-------+----+
|Offense_Category                        |Location_Name                     |count  |total  |percent|rank|
+----------------------------------------+----------------------------------+-------+-------+-------+----+
|Arson                                   |Residence/Home                    |7302   |11301  |64.61  |1   |
|Assault Offenses                        |Residence/Home                    |2134397|3593276|59.4   |1   |
|Bribery                                 |Residence/Home                    |476    |969    |49.12  |1   |
|Burglary/Breaking & Entering            |Residence/Home                    |196511 |272224 |72.19  |1   |
|Counterfeiting/Forgery                  |Residence/Home                    |13562  |41947  |32.33  |1   |
|Destruction/Damage/Vandalism of Property|Residence/Home                    |376088 |674319 |55.77  |1   |
|Embezzlement                        

- Homes are the primary crime setting.
- Public spaces (streets, alleys, highways) are prominent for crimes like trafficking and stolen property, suggesting the need for urban surveillance and law enforcement patrols.